In [ ]:
!git clone --recursive https://github.com/facebookresearch/NSVF.git

%cd /content/NSVF

!pip install -r requirements.txt

# %cd NSVF
# !pip install --editable ./

fatal: destination path 'NSVF' already exists and is not an empty directory.
/content/NSVF
  Cloning https://github.com/MultiPath/fairseq-stable.git to /tmp/pip-req-build-c9ec8mzg
  Running command git clone -q https://github.com/MultiPath/fairseq-stable.git /tmp/pip-req-build-c9ec8mzg
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/MultiPath/lpips-pytorch.git to /tmp/pip-req-build-71_uqb63
  Running command git clone -q https://github.com/MultiPath/lpips-pytorch.git /tmp/pip-req-build-71_uqb63
  Running command git submodule update --init --recursive -q


In [ ]:
!python setup.py build_ext --inplace

running build_ext
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:370: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
building 'fairnr.clib._ext' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/fairnr
creating build/temp.linux-x86_64-3.7/fairnr/clib
creating build/temp.linux-x86_64-3.7/fairnr/clib/src
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/torch/include -I/usr/local/lib/python3.7/dist-packag

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import os
os.environ['DATASET'] = "/content/drive/MyDrive/nsvf/dragonfly"
os.environ['SAVE'] = "/content/drive/MyDrive/nsvf/dragonfly/data"
# os.environ['DATASET'] = "/content/drive/MyDrive/Download/Spaceship"
# os.environ['SAVE'] = "/content/drive/MyDrive/Download"
%cd /content/NSVF

!python -u train.py ${DATASET} \
    --user-dir fairnr \
    --task single_object_rendering \
    --train-views "0..150" --view-resolution "1080x1080" \
    --max-sentences 1 --view-per-batch 2 --pixel-per-view 1024 \
    --no-preload \
    --sampling-on-mask 1.0 --no-sampling-at-reader \
    --valid-views "150..200" --valid-view-resolution "540x540" \
    --valid-view-per-batch 1 \
    --transparent-background "1.0,1.0,1.0" --background-stop-gradient \
    --arch nsvf_base \
    --initial-boundingbox ${DATASET}/bbox.txt \
    --use-octree \
    --raymarching-stepsize-ratio 0.125 \
    --discrete-regularization \
    --color-weight 128.0 --alpha-weight 1.0 \
    --optimizer "adam" --adam-betas "(0.9, 0.999)" \
    --lr 0.001 --lr-scheduler "polynomial_decay" --total-num-update 100000 \
    --criterion "srn_loss" --clip-norm 0.0 \
    --num-workers 0 \
    --seed 2 \
    --save-interval-updates 500 --max-update 100000 \
    --virtual-epoch-steps 5000 --save-interval 1 \
    --half-voxel-size-at  "5000,25000,75000" \
    --reduce-step-size-at "5000,25000,75000" \
    --pruning-every-steps 2500 \
    --keep-interval-updates 5 --keep-last-epochs 5 \
    --log-format simple --log-interval 1 \
    --save-dir ${SAVE} \
    --tensorboard-logdir ${SAVE}/tensorboard \
    | tee -a $SAVE/train.log

Streaming output truncated to the last 5000 lines.
|       from small pool |       8 MB |      77 MB |    1180 GB |    1180 GB |
|---------------------------------------------------------------------------|
| Active memory         |    6614 MB |   12046 MB |  314768 GB |  314762 GB |
|       from large pool |    6605 MB |   12037 MB |  313587 GB |  313581 GB |
|       from small pool |       8 MB |      77 MB |    1180 GB |    1180 GB |
|---------------------------------------------------------------------------|
| GPU reserved memory   |   12300 MB |   13304 MB |   40464 GB |   40452 GB |
|       from large pool |   12280 MB |   13222 MB |   40463 GB |   40451 GB |
|       from small pool |      20 MB |      82 MB |       0 GB |       0 GB |
|---------------------------------------------------------------------------|
| Non-releasable memory |    2085 MB |    5845 MB |  213549 GB |  213547 GB |
|       from large pool |    2074 MB |    5835 MB |  212317 GB |  212315 GB |
|       from 

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/nsvf/ac_nsvf/data"

# 3D reconstraction

In [ ]:
%cd /content/NSVF
import os
# os.environ['MODEL_PATH'] = "/content/drive/MyDrive/Download/checkpoint_last.pt"
# os.environ['SAVE'] = "/content/drive/MyDrive/Download/Spaceship"
os.environ['MODEL_PATH'] = "/content/drive/MyDrive/nsvf/dragonfly/data/checkpoint_last.pt"
os.environ['SAVE'] = "/content/drive/MyDrive/nsvf/dragonfly/"
os.environ['NAME'] = "dragonfly"
!python extract.py \
    --user-dir fairnr \
    --path ${MODEL_PATH} \
    --output ${SAVE} \
    --name ${NAME} \
    --format 'mc_mesh' \
    --mc-threshold 0.6 \
    --mc-num-samples-per-halfvoxel 5

/content/NSVF
2021-07-29 12:15:56 | INFO | fairnr_cli.extract | Namespace(bf16=False, cpu=False, format='mc_mesh', mc_num_samples_per_halfvoxel=5, mc_threshold=0.6, name='dragonfly', no_seed_provided=True, output='/content/drive/MyDrive/nsvf/dragonfly/', path='/content/drive/MyDrive/nsvf/dragonfly/data/checkpoint_last.pt', savetext=False, seed=1, tpu=False, user_dir='fairnr')
2021-07-29 12:15:59 | INFO | fairnr.modules.encoder | loaded 3969 voxel centers, 4840 voxel corners
2021-07-29 12:16:05 | INFO | fairnr.modules.encoder | marching cube...
2021-07-29 12:16:05 | INFO | fairnr.modules.encoder | reset chache
Building EasyOctree done. total #nodes = 67703, terminal #nodes = 30488 (time taken 15.304386 s)
tcmalloc: large alloc 3015680000 bytes == 0x55ac9411a000 @  0x7f5c3e0cfb6b 0x7f5c3e0ef379 0x7f5bd763626e 0x7f5bd76379e2 0x7f5c1bd52a73 0x7f5c1b288b7b 0x7f5c1b9e3bef 0x7f5c1b9c8480 0x7f5c1b5d1454 0x7f5c1b27c1c8 0x7f5c1bb56310 0x7f5c1bd21217 0x7f5c2dc999ae 0x7f5c2dde59de 0x55ac20fb8010 0